<a href="https://colab.research.google.com/github/lukkychan/resolution_changer/blob/main/Welcome_to_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install pytube

In [3]:
!mkdir Data
!mkdir -p Data/Videos/
!mkdir -p Data/frames/720p
!mkdir -p Data/frames/360p

In [ ]:
import pytube


# Specify the YouTube video URL
video_url = "https://www.youtube.com/watch?v=O5LW6HABcRA&pp=ygUQcmFkaGVzaHlhbSBzb25ncw%3D%3D"

# Create a YouTube object to access video details
youtube = pytube.YouTube(video_url)

# Get the available video streams
video_streams = youtube.streams.filter(file_extension='mp4').all()

# Print the available video streams and their resolutions
for i, stream in enumerate(video_streams):
    print(f"{i+1}. Resolution: {stream.resolution}, Format: {stream.mime_type}")

# Specify the desired resolution (e.g., "720p")
desired_resolution = "360p"

# Find the stream with the desired resolution
selected_stream = None
for stream in video_streams:
    if stream.resolution == desired_resolution:
        selected_stream = stream
        break

# Download the video with the selected stream
if selected_stream is not None:
    print(f"\nDownloading video with {selected_stream.resolution} resolution...")
    selected_stream.download(output_path="/content/Data/Videos/")
    print("Video downloaded successfully!")
else:
    print("Desired resolution not found. Please select another resolution.")

In [7]:
import cv2

# Specify the path to the video file
video_path = "/content/Data/Videos/720p.mp4"

# Specify the output folder to save the frames
output_folder = "/content/Data/frames/720p/"

# Specify the maximum number of frames to extract
max_frames = 150

# Open the video file
video = cv2.VideoCapture(video_path)

# Get the total number of frames in the video
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

# Calculate the frame interval to evenly sample the frames
frame_interval = max(total_frames // max_frames, 1)

# Initialize a counter to keep track of the extracted frames
frame_count = 0

# Loop through the frames and extract the desired number of frames
while frame_count < max_frames:
    # Read the current frame
    ret, frame = video.read()

    # Check if the frame was successfully read
    if not ret:
        break

    # Save the frame as an image file
    frame_path = f"{output_folder}frame_{frame_count}.jpg"
    cv2.imwrite(frame_path, frame)

    # Increment the frame count
    frame_count += 1

    # Move to the next frame based on the frame interval
    video.set(cv2.CAP_PROP_POS_FRAMES, frame_count * frame_interval)

# Release the video capture object
video.release()

In [10]:
from PIL import Image
import os

def get_image_size(image_path):
    # Check if the file exists
    if not os.path.exists(image_path):
        print(f"File '{image_path}' does not exist.")
        return None

    try:
        with Image.open(image_path) as img:
            width, height = img.size
            return width, height
    except Exception as e:
        print(f"Error occurred while opening the image: {e}")
        return None

# Example usage
image_path = "/content/Data/frames/360p/frame_0.jpg"  # Replace with the actual path to your image file
image_size = get_image_size(image_path)

if image_size:
    print(f"The image size is: {image_size[0]}x{image_size[1]} pixels.")


The image size is: 640x360 pixels.


In [ ]:
!git clone https://github.com/AarohiSingla/SRGAN_CustomDataset.git

In [11]:
!python /content/SRGAN_CustomDataset/main.py --LR_path /content/Data/frames/360p/ --GT_path /content/Data/frames/720p/

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Traceback (most recent call last):
  File "/content/SRGAN_CustomDataset/main.py", line 1, in <module>
    from mode import *
  File "/content/SRGAN_CustomDataset/mode.py", line 13, in <module>
    from skimage.measure import compare_psnr
ImportError: cannot import name 'compare_psnr' from 'skimage.measure' (/usr/local/lib/python3.10/dist-packages/skimage/measure/__init__.py)
